In [4]:
from data.DeepCDRDataModule import DeepCDRDataModule
from models.DeepCDRLitModule import DeepCDRLitModule
from util.config import *
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Configuration For Training

In [9]:
dpath = 'data/raw'
drug_info_file = '%s/GDSC/1.Drug_listMon Jun 24 09_00_55 2019.csv' % dpath
cell_line_info_file = '%s/CCLE/Cell_lines_annotations_20181226.txt' % dpath
drug_feature_file = '%s/GDSC/drug_graph_feat' % dpath
genomic_mutation_file = '%s/CCLE/genomic_mutation_34673_demap_features.csv' % dpath
cancer_response_exp_file = '%s/CCLE/GDSC_IC50.csv' % dpath
gene_expression_file = '%s/CCLE/genomic_expression_561celllines_697genes_demap_features.csv' % dpath
methylation_file = '%s/CCLE/genomic_methylation_561celllines_808genes_demap_features.csv' % dpath
raw_files = NeededFiles(drug_info_file, cell_line_info_file, drug_feature_file, genomic_mutation_file,
                        cancer_response_exp_file, gene_expression_file, methylation_file)

GCN = ConvType.GCN
GAT = ConvType.GAT
tanh = ActivationFunc.Tanh
relu = ActivationFunc.ReLU
f_tanh = FuncActivationFunc.Tanh
f_relu = FuncActivationFunc.ReLU

drug_config = GNNConfig([75, 256, 256, 256], [GAT, GAT, GAT], [f_relu, f_relu, f_relu])
mutation_config = Conv1DConfig([1, 5, 10], [700, 5], [tanh, relu], [10, 10])
g_expr_config = FCConfig([256], [tanh], output=100)
meth_config = FCConfig([256], [tanh], output=100)
regressor_cnn_config = Conv1DConfig([1, 10, 5, 5, 5], [150, 5, 5, 5], [relu, relu, relu, relu], [2, 3, 3, 3])
regressor_fc_config = FCConfig([], [], output=1)
config = DeepCDRConfig(drug_config, mutation_config, g_expr_config, meth_config, regressor_cnn_config, regressor_fc_config, 0.1, False)

In [10]:
data = DeepCDRDataModule(raw_files, 100, batch_size=10)
model = DeepCDRLitModule(config)

/home/user/PycharmProjects/DeepCDR/venv/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


## Training The Model

In [11]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.lr_monitor import LearningRateMonitor

pl.seed_everything(0)
early_stopping = EarlyStopping(monitor="Validation loss", mode='min', stopping_threshold=0.005, patience=8)
logger=TensorBoardLogger(save_dir="log", name="DeepCDR_Logs", version="GAT Test")
trainer = pl.Trainer(logger=logger, callbacks=[early_stopping], log_every_n_steps=1)
debug_trainer = pl.Trainer(overfit_batches=1, logger=logger, max_epochs=100, log_every_n_steps=1)

Global seed set to 0
/home/user/PycharmProjects/DeepCDR/venv/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.


In [12]:
trainer.fit(model, datamodule=data)


  | Name              | Type              | Params | In sizes              | Out sizes 
---------------------------------------------------------------------------------------------
0 | drug_gnn          | Sequential_542761 | 152 K  | [[100, 75], [2, 101]] | [100, 256]
1 | mutation_cnn1d    | Sequential        | 342 K  | [1, 1, 34673]         | [1, 100]  
2 | g_expr_fcn        | Sequential        | 204 K  | [1, 697]              | [1, 100]  
3 | meth_fcn          | Sequential        | 232 K  | [1, 808]              | [1, 100]  
4 | regression_conv1d | Sequential        | 239 K  | [1, 1, 25900]         | [1, 100]  
5 | regression_fc     | Sequential        | 101    | [1, 100]              | [1, 1]    
---------------------------------------------------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.687     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

## Evaluation

In [14]:
# Loading TensorBoard
%reload_ext tensorboard
%tensorboard --logdir log/

Launching TensorBoard...

In [18]:
gat_model = DeepCDRLitModule.load_from_checkpoint('log/DeepCDR_Logs/GAT Test/checkpoints/epoch=12-step=78.ckpt')
gcn_model = DeepCDRLitModule.load_from_checkpoint('log/DeepCDR_Logs/GCN Test/checkpoints/epoch=8-step=54.ckpt')
print("Testing GAT Model")
trainer.test(gat_model, data)
print("Testing GCN Model")
trainer.test(gcn_model, data)

Testing GAT Model


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        Test loss            1.081456184387207
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Testing GCN Model


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        Test loss           1.1563169956207275
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'Test loss': 1.1563169956207275}]